# Chat with prompty

## 0. Install dependent packages

In [ ]:
%%capture --no-stderr
%pip install promptflow-devkit

In [ ]:
%pip install promptflow promptflow-tools

## 1. Prompty:

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [ ]:
with open("prompty/chat.prompty") as fin:
    print(fin.read())

### Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

We need to set up the connection if we haven't added it before. After created, it's stored in local db and can be used in any flow.

Prepare your Azure Open AI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

In [ ]:
#%pip install keyrings.alt

In [ ]:
from promptflow.client import PFClient
from promptflow.connections import AzureOpenAIConnection

client = PFClient()

connection = AzureOpenAIConnection(
    name="my_azure_open_ai_connection",
    api_key="YOUR_API_KEY",
    api_base="https://YOUR_RESOURCE_NAME.openai.azure.com"
)

result = client.connections.create_or_update(connection)
print("Connection created or updated:", result)


In [ ]:
conn_name = "my_azure_open_ai_connection"
conn = client.connections.get(name=conn_name)
print("using this connection :",conn_name)

### Execute prompty as function

In [ ]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load("prompty/chat.prompty")
# execute the flow as function
question = "What is the capital of France?"
result = f(question=question)
result

You can override connection with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [ ]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration


# override configuration with created connection in AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    connection="my_azure_open_ai_connection", azure_deployment="gpt-4o"
)

# override openai connection with OpenAIModelConfiguration
# configuration = OpenAIModelConfiguration(
#     connection=connection,
#     model="gpt-3.5-turbo"
# )

override_model = {
    "configuration": configuration,
}

# load prompty as a flow
f = Prompty.load("prompty/chat.prompty", model=override_model)
# execute the flow as function
question = "What is the capital of France?"
result = f(question=question)
result

### Visualize trace by using start_trace

In [ ]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Re-run below cell will collect a trace in trace UI.

In [ ]:
# rerun the function, which will be recorded in the trace
result = f(question=question)
result

### Eval the result 

In this example, we will use a prompt that determines whether a chat conversation contains an apology from the assistant.

In [ ]:
eval_prompty = "prompty/apology.prompty"

with open(eval_prompty) as fin:
    print(fin.read())

Note: the eval flow returns a `json_object`.

In [ ]:
# load prompty as a flow
eval_flow = Prompty.load(eval_prompty)
# execute the flow as function
result = eval_flow(question=question, answer=result, messages=[])
result

In [ ]:
!pf service stop

# Batch run with multi-line data


In [ ]:
from promptflow.client import PFClient

flow = "./prompty/chat.prompty"  # path to the prompty file
data = "./prompty/data.jsonl"  # path to the data file

# create run with the flow and data
pf = PFClient()
base_run = pf.run(
    flow=flow,
    data=data,
    column_mapping={
        "question": "${data.question}",
        "chat_history": "${data.chat_history}",
    },
    stream=True,
)

In [ ]:
details = pf.get_details(base_run)
details

In [ ]:
details.head(3)['outputs.output'].values